In [2]:
print("Please check to create the DB with Beev Electic Vehicle Specs Data")
print("Alex, please answer")

Please check to create the DB with Beev Electic Vehicle Specs Data
Alex, please answer


In [3]:
#LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Graphics
import matplotlib.pyplot as plt
import seaborn as sns
#KNeighbors
from sklearn.neighbors import NearestNeighbors
#Scalers
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#Data selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
#METRICS
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score,accuracy_score
from sklearn.metrics import mean_squared_error
#RANDOM
import random as rd

In [4]:
df = pd.read_csv("Beev Electric Vehicle Specs Data.csv")

In [40]:
df.columns

Index(['Full Name', 'Model', 'Brand', 'Acceleration (0-100 km/h)',
       'Range (km)', 'Average Winter Range', 'Highway Winter Range',
       'City Winter Range', 'Average Summer Range', 'Highway Summer Range',
       'City Summer Range', 'Optimal Slow Charge', 'Optimal Fast Charge',
       'Battery Capacity (kW)', 'Useable Battery Capacity', 'Category',
       'Government Incentive Category for Help', 'Vehicle Consumption (Wh)',
       'Average Winter Consumption', 'Highway Winter Consumption',
       'City Winter Consumption', 'Average Summer Consumption',
       'Highway Summer Consumption', 'City Summer Consumption', 'Torque',
       'Availability - deleted', 'Distance', 'Main Finishes', 'Finishes',
       'Height (mm)', 'Main Image', 'Product Image', 'Images', 'Width',
       'Last Edition of Finishes Date', 'Length (mm)', 'Label',
       'Average Range', 'Seats', 'Empty Weight (kg)', 'Max Price', 'Min Price',
       'Slow Charge Port', 'Fast Charge Port', 'Main Price',
       'M

In [6]:
nan_values = df.isna().sum()

In [28]:
#New Dataframe with just columns I thought were good idea to display. 
#However, I already put on trello a question so we know which information to display at the end
df_new = pd.DataFrame(zip(df['Full Name'],df['Brand'], df['Model'], df['Main Price'], df['Category']))

In [29]:
#Renaming the columns to the correct name
df_new.rename(columns = {0:'Full Name', 1:'Brand', 2:'Model', 3:'Main Price',4: 'Category'}, inplace = True)

In [30]:
df_new['Main Price'].value_counts(dropna = False)   # We either drop the Nans or we fill with the average
df_new['Main Price'].fillna(df_new['Main Price'].mean(), inplace = True)  #The mean is ok since thre are very few values. But giving a wrong price for even 1 care can be really bad

In [31]:
df_new['Category'].value_counts(dropna = False)         #I would say we can use get_dummies for the Category so we can use it as X?
df_new = pd.get_dummies(df_new, columns=['Category'])   #The thing about the dummies, is, how will the user input the correct Category?
#An idea would be to rename the columns to just for example: 'SUV' or 'Citadine'

In [32]:
df_new.columns                                        

Index(['Full Name', 'Brand', 'Model', 'Main Price', 'Category_Berline',
       'Category_Citadine', 'Category_Compacte', 'Category_SUV',
       'Category_Utilitaire', 'Category_Van'],
      dtype='object')

In [33]:
df_new.head()

,Full Name,Brand,Model,Main Price,Category_Berline,Category_Citadine,Category_Compacte,Category_SUV,Category_Utilitaire,Category_Van
0,Aiways U5,Aiways,U5,39300.0,0,0,0,1,0,0
1,Audi e-tron 50 quattro,Audi,e-tron 50,71900.0,0,0,0,1,0,0
2,Audi e-tron 55 quattro,Audi,e-tron 55,84200.0,0,0,0,1,0,0
3,Audi e-tron GT quattro,Audi,e-tron GT quattro,103040.0,1,0,0,0,0,0
4,Audi e-tron GT RS,Audi,e-tron GT RS,142830.0,1,0,0,0,0,0


In [34]:
X = df_new[['Main Price']]          
distanceKNN = NearestNeighbors(n_neighbors=3).fit(X)

In [35]:
distanceKNN.kneighbors([[60000]], 3, return_distance = False)

/Users/cedricpinel/opt/anaconda3/envs/Wild_code/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


array([[148,  16,  57]])

In [36]:
df[['Full Name', 'Main Price']].iloc[[148, 16, 57]]  #This was a try with just the main price set at 60000. It's a pretty great result

,Full Name,Main Price
148,Volvo C40 Recharge Twin,59760.0
16,BMW i4 eDrive40,59700.0
57,Mercedes EQA 350 4MATIC,60700.0


In [38]:
#Fit model2 with the getdummies added in X2
X2 = df_new[['Main Price','Category_Berline','Category_Citadine','Category_Compacte','Category_SUV','Category_Utilitaire','Category_Van']]
model2 = NearestNeighbors(n_neighbors=3).fit(X2)


In [39]:
model2.kneighbors([[60000,0,0,0,1,0,0]], 3, return_distance = False) # checking the result for a SUV category with 60000€

/Users/cedricpinel/opt/anaconda3/envs/Wild_code/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


array([[148,  16,  57]])

In [41]:
df[['Full Name', 'Main Price', 'Category']].iloc[[148, 16, 57]] # in the result two are SUV

,Full Name,Main Price,Category
148,Volvo C40 Recharge Twin,59760.0,SUV
16,BMW i4 eDrive40,59700.0,Berline
57,Mercedes EQA 350 4MATIC,60700.0,SUV


In [42]:
model2.kneighbors([[60000,0,0,0,0,1,0]], 3, return_distance = False) # checking the result for a Utilitaire category with 60000€

/Users/cedricpinel/opt/anaconda3/envs/Wild_code/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


array([[148,  16,  57]])

In [ ]:
#The result is the same so we will have to filter, fit and Kneighbors I think
#In this case I am not sure that get dummies will help us